In [3]:
from numpy import *
import matplotlib.pyplot as plt 
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn import preprocessing
import pandas as pd

In [4]:
dataset = pd.read_csv('/content/drive/MyDrive/homeprices.csv')
dataset.head()

,area,price
0,2600,550000
1,3000,565000
2,3200,610000
3,3600,680000
4,4000,725000


In [5]:
#Independent variable
X = dataset.iloc[:,[0]].values 
X

array([[2600],
       [3000],
       [3200],
       [3600],
       [4000]])

In [6]:
#Dependent variable
Y = dataset.iloc[:,[1]].values
Y

array([[550000],
       [565000],
       [610000],
       [680000],
       [725000]])

In [7]:
#Mean of our input
x_mean = np.mean(X)
x_mean

3280.0

In [8]:
 y_mean = np.mean(Y)
 y_mean

626000.0

In [9]:
n = len(X)
numerator = 0
denominator = 0

for i in range(n):
    numerator += (X[i] - x_mean) * (Y[i] - y_mean)
    denominator += (X[i] - x_mean) ** 2


In [10]:
b1 = numerator / denominator
b0 = y_mean - (b1 * x_mean)
print(b1)
print(b0)





[135.78767123]
[180616.43835616]


In [11]:
X = 3300
y_pred = b0 + b1 * X
y_pred

array([628715.75342466])

In [12]:
X = 5000
y_pred = b0 + b1 * X
y_pred

array([859554.79452055])

In [13]:
root_mean_sq_err = 0
for i in range(n):
    y_pred = b0 + b1*X[i]
    root_mean_sq_err += (Y[i] - y_pred) ** 2

root_mean_sq_err = np.sqrt(root_mean_sq_err/n)
print('Root Mean Squares Error == ',str(root_mean_sq_err))

TypeError: ignored

# Evaluation of Best Classification Model Using GridSearchCV

In [16]:
df = pd.read_csv("/content/drive/MyDrive/My Colab Notebook/NoSqli_Dataset.csv")
df.head()

,Feature0,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,ClassLabel
0,0,0,0,1,1,0,0,0,0,0,0
1,1,1,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0


In [17]:

X = df.drop(['ClassLabel'],axis='columns')
Y = df['ClassLabel']

In [18]:
Y.value_counts()

0    801
1    203
Name: ClassLabel, dtype: int64

In [19]:
X.head(2)

,Feature0,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9
0,0,0,0,1,1,0,0,0,0,0
1,1,1,0,1,0,0,0,0,0,0


In [20]:
Y.head(5)

0    0
1    1
2    0
3    0
4    0
Name: ClassLabel, dtype: int64

In [21]:
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix , classification_report

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear','poly']
        }  
    },
    'decision_tree': {
        'model': tree.DecisionTreeClassifier(),
        'params' : {
            'criterion':["gini","entropy"],
            'max_depth':[1,2,3,4,5,6,7,None]
        }
    },
    'Adaboost':{
        'model':AdaBoostClassifier(base_estimator = tree.DecisionTreeClassifier(random_state = 11, max_features = "auto",max_depth = None)),
        'params':{
              'base_estimator__criterion' : ["gini", "entropy"],
              'base_estimator__splitter' :   ["best", "random"],
              'n_estimators': [1, 2]
             }
    },

    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    },
    'Naive_Bayes:BernoulliNB':{
        'model':BernoulliNB(),
        'params':{
            'alpha': [0.01, 0.1, 0.5, 1.0, 10.0],
        }
    },


}

In [36]:
scores = []

ConfusionMatrix={}
for model_name, mp in model_params.items():
    grid =  GridSearchCV(mp['model'],param_grid= mp['params'], cv=5, return_train_score=False,n_jobs=-1)
    grid.fit(X, Y)
    y_pred=grid.best_estimator_.predict(X)
    ConfusionMatrix[model_name]=confusion_matrix(Y, y_pred)
    scores.append({
        'model': model_name,
        'best_score': grid.best_score_,
        'best_params': grid.best_params_,
        
    
    })


    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.918343,"{'C': 20, 'kernel': 'poly'}"
1,decision_tree,0.914358,"{'criterion': 'entropy', 'max_depth': None}"
2,Adaboost,0.916348,"{'base_estimator__criterion': 'gini', 'base_es..."
3,random_forest,0.913358,{'n_estimators': 5}
4,logistic_regression,0.904398,{'C': 1}
5,Naive_Bayes:BernoulliNB,0.878517,{'alpha': 1.0}


In [38]:
import seaborn as sn
%matplotlib inline
import matplotlib.pyplot as plt


for model_name,matrix in ConfusionMatrix.items():
    print(model_name)
    plt.figure(figsize = (10,7))
    sn.heatmap(cm, annot=True)
    plt.xlabel('Predicted')
    plt.ylabel('Truth')


svm
[[796   5]
 [ 57 146]]
decision_tree
[[797   4]
 [ 51 152]]
Adaboost
[[797   4]
 [ 51 152]]
random_forest
[[796   5]
 [ 52 151]]
logistic_regression
[[791  10]
 [ 67 136]]
Naive_Bayes:BernoulliNB
[[770  31]
 [ 44 159]]


# Logistic Regression from scratch

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1234)

In [24]:
class LogisticRegression:

    def __init__(self, learning_rate=0.001, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # init parameters
        self.weights = np.zeros(n_features)
        self.bias = 0

        # gradient descent
        for _ in range(self.n_iters):
            # approximate y with linear combination of weights and x, plus bias
            
            linear_model = np.dot(X, self.weights) + self.bias  #y = wx+b
            # apply sigmoid function
            y_predicted = self._sigmoid(linear_model)                            #  0 ->1     0 or 1
 
            # compute gradients
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)
            # update parameters
            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self._sigmoid(linear_model)
        y_predicted_cls = [1 if i > 0.5 else 0 for i in y_predicted]
        return np.array(y_predicted_cls)

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

In [25]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy


regressor = LogisticRegression(learning_rate=0.0001, n_iters=1000)
regressor.fit(X_train, y_train)
predictions = regressor.predict(X_test)

print("LR classification accuracy:", accuracy(y_test, predictions))

LR classification accuracy: 0.8407960199004975
